In [6]:
import numpy as np
from matplotlib import pyplot as plt
import h5py
import glob
import import_ipynb
#Import file that reads the .hdf5 files
import read_ligo

This assignment focuses on finding gravitational waves. This file will be the main, where I will use functions from other jupyter notebooks to keep everything organized. Let's get started by loading all of the data files.

In [20]:
#Define file path location
filepath = 'LOSC_Event_tutorial/LOSC_Event_tutorial/'

#I will denote Hanford data with '_H' and Livingston data with '_L'
#Create an array of the files of interest
files_H  = ['H-H1_LOSC_4_V2-1126259446-32.hdf5','H-H1_LOSC_4_V2-1128678884-32.hdf5',
            'H-H1_LOSC_4_V2-1135136334-32.hdf5','H-H1_LOSC_4_V1-1167559920-32.hdf5'] 
files_L  = ['L-L1_LOSC_4_V2-1126259446-32.hdf5','L-L1_LOSC_4_V2-1128678884-32.hdf5',
            'L-L1_LOSC_4_V2-1135136334-32.hdf5','L-L1_LOSC_4_V1-1167559920-32.hdf5'] 

#Initialize the strains
strains_H, strains_L = np.zeros([len(files_H), 131072]), np.zeros([len(files_L), 131072])

#Initialize utc
utc_H, utc_L = [], []

#Create array with all the filters files and the filters at Hanford and Livingston
templates = np.array(['GW150914_4_template.hdf5','LVT151012_4_template.hdf5',
                      'GW151226_4_template.hdf5','GW170104_4_template.hdf5'])
t_H, t_L = np.zeros([len(templates), 131072]), np.zeros([len(templates), 131072])

#Loop over the files_H and files_L to save all of the strains utcs, dts, filters, and templates
for i in range(len(files_H)):
    print('Reading files:', files_H[i], 'and', files_L[i])
    strains_H[i,:], dt, utc_1 = read_ligo.read_file(filepath+files_H[i])
    strains_L[i,:], dt, utc_2 = read_ligo.read_file(filepath+files_L[i])
    utc_H.append(utc_1)
    utc_L.append(utc_2)
    t_H[i:,],t_L[i:,] = read_ligo.read_template(filepath+templates[i])
    
#Turn utcs into arrays for later
np.array(utc_H)
np.array(utc_L)

#Define event names
events = ['GW150914','LVT151012','GW151226','GW170104']

Reading files: H-H1_LOSC_4_V2-1126259446-32.hdf5 and L-L1_LOSC_4_V2-1126259446-32.hdf5
Reading files: H-H1_LOSC_4_V2-1128678884-32.hdf5 and L-L1_LOSC_4_V2-1128678884-32.hdf5
Reading files: H-H1_LOSC_4_V2-1135136334-32.hdf5 and L-L1_LOSC_4_V2-1135136334-32.hdf5
Reading files: H-H1_LOSC_4_V1-1167559920-32.hdf5 and L-L1_LOSC_4_V1-1167559920-32.hdf5


### Question 1 a)